<a href="https://colab.research.google.com/github/Sidd-Shanmuhavel/Python---WebCrawler-Using-BeautifulSoup4/blob/master/Monster_com_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import csv
import json
import requests
import pandas as pd

In [80]:
role = "data-analyst"
link = f'https://www.monster.ie/jobs/search/?q={role}'
print(link)

https://www.monster.ie/jobs/search/?q=data-analyst


In [7]:
source = requests.get(link).text
soup = BeautifulSoup(source,'lxml')

#fetching the search results
job_body = soup.find(id = 'ResultsContainer')

#It returns a json object
script = job_body.find('script').text

#Parsing the json, the result is a dict
jsonValue = json.loads(script)

In [12]:
#Links of all the jobs appearing
link_list = []
for job in jsonValue['itemListElement']:
    url = job['url']
    if len(url) > 0:
        link_list.append(url)

In [59]:
#Location of all the jobs posted
location_list = []
role_name_list = []
salary_list = []
for link in link_list:
    one_job = requests.get(link).text
    job_soup = BeautifulSoup(one_job,'lxml')
    heading = job_soup.find('div',class_='heading')

    #Location
    location_list.append(heading.find(class_ = 'location c-gray-6').text)
    
    #Role Name
    role_name_list.append(job_soup.find('div',class_='heading').h1.text)

    #Salary
    about_job = job_soup.find('div', class_='col-md-12 col-8 ta-md-l ta-r font-semibold')
    salary_list.append(about_job.text)

In [69]:
#Checking if the Salary is mentioned in the posting or nots
for index in range(len(salary_list)):
  if any(map(str.isdigit, salary_list[index])) == False:
    salary_list[index] = 'Salary Not Quoted'

In [71]:
#creating a dictionary of the data collected
data = {'Role':role_name_list,'Location':location_list,'Salary':salary_list,'Job Link': link_list}

In [72]:
#creating a data frame with the data scrapped
df = pd.DataFrame(data)
df.head()

,Role,Location,Salary,Job Link
0,Data Analyst/ Developer,"Dublin, Dublin",€50000 - €60000 per year,https://job-openings.monster.ie/data-analyst-d...
1,Master Data Governance Analyst,"Westmeath, Westmeath",€60000 - €70000 per year,https://job-openings.monster.ie/master-data-go...
2,Senior Data Analyst - Investments,"Dublin, Dublin",Salary Not Quoted,https://job-openings.monster.ie/senior-data-an...
3,Data Analyst - Funds - 50-55k,"Dublin, Dublin",€50000 - €55000 per year,https://job-openings.monster.ie/data-analyst-f...
4,Investment Data Analyst,"Dublin, Dublin",€45000 - €60000 per year,https://job-openings.monster.ie/investment-dat...
